A partir de arquivo que contém todos os CEPs presentes na massa dados, descobriremos as posições geográficas, latitude e longitude, de cada CEP e as guardaremos em um novo arquivo.

In [6]:
import pandas as pd
from geopy.geocoders import Nominatim
import datetime
import requests
from requests.adapters import HTTPAdapter, Retry
import time

In [7]:
df = pd.read_csv('arquivos/Endereços dos CEPs.csv',usecols=['cep'],dtype={'cep':'str'})
df.head()

,cep
0,60150060
1,60320105
2,60713480
3,60431075
4,60430660


Vamos definir uma função que retorna, em uma tupla, as coordenadas geográficas de um CEP, que deve ser informado em formato str.

In [22]:
s = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[ 502, 503, 504 ])
s.mount('http://', HTTPAdapter(max_retries=retries))

def coord_geog(cep):
    # Capturando endereço completo a partir do CEP
    url = 'https://viacep.com.br/ws/' + cep + '/json/'
    content = s.get(url)
    
    if content.status_code == 200:
        py_content = content.json()
        rua = py_content['logradouro']
        bairro = py_content['bairro']
        cidade = py_content['localidade']
    
        # Descobrindo coordenadas geográficas a partir do endereço completo
        locator = Nominatim(user_agent='TccLg')
        loc = locator.geocode(f"{rua}, {bairro}-{cidade}")
        if not loc == None:
            return loc.latitude,loc.longitude
        else:
            return None,None

Ao invés de utilizarmos diretamente a função apply do Pandas, trabalharemos com uma lista para poder mostrar em tela
o progresso da pesquisa dos CEPs.

In [9]:
ceps = df['cep'].to_list()
lats = list()
longs = list()

for cep in ceps:
    try:
        lat,long = coord_geog(cep)
        print(f"{datetime.datetime.now()}: Tratou {cep}: {lat} - {long}")
        time.sleep(5)
    except:
        print(f"{datetime.datetime.now()}: Erro com {cep}. Tentando novamente em 60 segundos.")
        time.sleep(60)
        lat,long = coord_geog(cep)
        print(f"{datetime.datetime.now()}: Tratou {cep}: {lat} - {long}")

    lats.append(lat)
    longs.append(long)

2022-02-28 19:25:36.373123: Tratou 60150060: -3.7363272 - -38.5048328
2022-02-28 19:25:42.522156: Tratou 60320105: -3.7267916 - -38.5628264
2022-02-28 19:25:48.865014: Tratou 60713480: -3.7956573 - -38.5783172
2022-02-28 19:25:54.906521: Tratou 60431075: -3.7396638 - -38.5501175
2022-02-28 19:26:00.947280: Tratou 60430660: -3.7400877 - -38.5464956
2022-02-28 19:26:07.398231: Tratou 60812475: -3.7741966500000004 - -38.45728332333569
2022-02-28 19:26:13.544630: Tratou 60740745: -3.77914 - -38.5550957
2022-02-28 19:26:19.890708: Tratou 60120001: -3.7380906 - -38.5077497
2022-02-28 19:26:26.341499: Tratou 60713300: -3.7999081 - -38.5806813
2022-02-28 19:26:32.485685: Tratou 60420312: -3.7646365 - -38.5433673
2022-02-28 19:26:38.631217: Tratou 60425784: -3.7486664 - -38.5460555
2022-02-28 19:26:44.775451: Tratou 60191335: -3.7499578 - -38.46664
2022-02-28 19:26:51.121733: Tratou 60176125: -3.7440767 - -38.4712426
2022-02-28 19:26:57.162512: Tratou 60822482: -3.7985946 - -38.4945333
2022-02-

2022-02-28 19:38:05.206753: Tratou 60135180: -3.7437919 - -38.5131123
2022-02-28 19:38:11.352857: Tratou 60135430: -3.7485753 - -38.4958539
2022-02-28 19:38:17.498657: Tratou 60150120: -3.7328235 - -38.5130713
2022-02-28 19:38:23.542163: Tratou 60120075: -3.7531133 - -38.517149
2022-02-28 19:38:29.585502: Tratou 60055265: -3.7415266 - -38.5247362
2022-02-28 19:38:37.187199: Tratou 60140110: -3.7378797 - -38.510039
2022-02-28 19:38:43.412567: Tratou 60410298: -3.7584911 - -38.5384701
2022-02-28 19:38:49.557616: Tratou 60020060: -3.7444673 - -38.5372105
2022-02-28 19:38:55.907635: Tratou 60140195: -3.7333686 - -38.5178876
2022-02-28 19:39:02.056247: Tratou 60115015: -3.7236926 - -38.5083333
2022-02-28 19:39:08.404691: Tratou 60864100: -3.7999081 - -38.5105011
2022-02-28 19:39:14.557988: Tratou 60310402: -3.7158776 - -38.5464361
2022-02-28 19:39:20.694105: Tratou 60011170: -3.7342785 - -38.5432127
2022-02-28 19:39:26.841887: Tratou 60050140: -3.7478909 - -38.5275049
2022-02-28 19:39:32.98

2022-02-28 19:50:21.673241: Tratou 60110261: -3.7248948 - -38.514122
2022-02-28 19:50:27.819429: Tratou 60421106: -3.7704827 - -38.557441
2022-02-28 19:50:33.962019: Tratou 60761485: -3.8095929 - -38.5600379
2022-02-28 19:50:40.106992: Tratou 60120055: -3.7534685 - -38.511554
2022-02-28 19:50:46.250423: Tratou 60110430: -3.7362426 - -38.5141496
2022-02-28 19:50:52.394772: Tratou 60347365: -3.7116127 - -38.5915239
2022-02-28 19:50:58.539415: Tratou 60430560: -3.7418283 - -38.5493815
2022-02-28 19:51:04.684740: Tratou 60120340: -3.7528582 - -38.514948
2022-02-28 19:51:11.136699: Tratou 60821220: -3.7846551 - -38.5042525
2022-02-28 19:51:17.281887: Tratou 60861710: -3.814425 - -38.5246454
2022-02-28 19:51:23.528933: Tratou 60130000: -3.7453569 - -38.5221511
2022-02-28 19:51:29.673278: Tratou 60040480: -3.7486937 - -38.529215
2022-02-28 19:51:36.020835: Tratou 60120002: -3.7455614 - -38.5101362
2022-02-28 19:51:42.165780: Tratou 60843040: -3.8336248 - -38.4748051
2022-02-28 19:51:48.309045

2022-02-28 20:02:51.577107: Tratou 60015340: -3.7390514 - -38.5391599
2022-02-28 20:02:57.723471: Tratou 60527400: -3.7488829 - -38.5931675
2022-02-28 20:03:03.867568: Tratou 60834375: -3.7907084 - -38.4769797
2022-02-28 20:03:10.010789: Tratou 60360840: -3.7452745 - -38.5919285
2022-02-28 20:03:16.157287: Tratou 60810001: -3.765491 - -38.4958403
2022-02-28 20:03:22.402943: Tratou 60750790: -3.8290123 - -38.5580773
2022-02-28 20:03:28.544537: Tratou 60441260: -3.7605293 - -38.5710693
2022-02-28 20:03:34.794680: Tratou 60182160: -3.7270765 - -38.4626592
2022-02-28 20:03:40.939781: Tratou 60191365: -3.7499393 - -38.4612634
2022-02-28 20:03:47.081847: Tratou 60822676: -3.7892897 - -38.4959552
2022-02-28 20:03:53.229368: Tratou 60714170: -3.783287 - -38.5658193
2022-02-28 20:03:59.572072: Tratou 60130301: -3.754642 - -38.5106145
2022-02-28 20:04:05.721077: Tratou 60192085: -3.74517 - -38.4814849
2022-02-28 20:04:11.864319: Tratou 60420430: -3.763291 - -38.5461259
2022-02-28 20:04:18.003427

2022-02-28 20:15:18.185141: Tratou 60411230: None - None
2022-02-28 20:15:24.463451: Tratou 60410322: -3.7674857 - -38.5389572
2022-02-28 20:15:30.574554: Tratou 60170151: -3.7352192 - -38.4947754
2022-02-28 20:15:36.718742: Tratou 60325610: -3.7277608 - -38.5536405
2022-02-28 20:15:42.864683: Tratou 60431072: -3.7396638 - -38.5501175
2022-02-28 20:15:49.005850: Tratou 61650350: None - None
2022-02-28 20:15:55.152026: Tratou 60450115: -3.7372691 - -38.5659733
2022-02-28 20:16:01.297380: Tratou 60824190: -3.8120014 - -38.4973208
2022-02-28 20:16:07.438849: Tratou 60135285: -3.7383646 - -38.5162043
2022-02-28 20:16:13.586298: Tratou 60410374: -3.7625836 - -38.5297015
2022-02-28 20:16:19.931396: Tratou 60821450: -3.7843973 - -38.4982377
2022-02-28 20:16:26.075356: Tratou 60115044: -3.7458526 - -38.5149792
2022-02-28 20:16:32.219394: Tratou 60125101: None - None
2022-02-28 20:16:38.365652: Tratou 60810025: -3.7632134 - -38.4844308
2022-02-28 20:16:44.712292: Tratou 60115001: -3.7439217 - -

2022-02-28 20:28:09.262824: Tratou 60420470: -3.770043 - -38.5435247
2022-02-28 20:28:15.407904: Tratou 60541706: -3.7791541 - -38.5908012
2022-02-28 20:28:21.554921: Tratou 60810050: -3.7604416 - -38.4905478
2022-02-28 20:28:27.696138: Tratou 60430160: -3.7477813 - -38.5519241
2022-02-28 20:28:33.846882: Tratou 60181032: -3.7227744 - -38.4774651
2022-02-28 20:28:40.187301: Tratou 60420570: -3.7652722 - -38.5478401
2022-02-28 20:28:51.248334: Tratou 60873010: -3.8549832 - -38.5007332
2022-02-28 20:28:57.391163: Tratou 60812450: -3.7742369 - -38.4606651
2022-02-28 20:29:03.537541: Tratou 60135208: -3.7470392 - -38.5061413
2022-02-28 20:29:09.683048: Tratou 60710710: -3.7939389 - -38.5645164
2022-02-28 20:29:16.146673: Tratou 60822610: -3.7900305 - -38.4997165
2022-02-28 20:29:22.639379: Tratou 60712092: -3.7997689 - -38.5663019
2022-02-28 20:29:28.828240: Tratou 60870576: -3.8569903 - -38.5070875
2022-02-28 20:29:34.972905: Tratou 60130175: None - None
2022-02-28 20:29:41.117086: Tratou

2022-02-28 20:41:17.954571: Tratou 60834005: -3.7877026 - -38.4605586
2022-02-28 20:41:24.303606: Tratou 60455473: -3.7416669 - -38.5650748
2022-02-28 20:41:30.541280: Tratou 60165081: -3.7272662 - -38.4955482
2022-02-28 20:41:36.711279: Tratou 60531800: -3.7740117 - -38.6038971
2022-02-28 20:41:43.130318: Tratou 60762060: -3.8102971 - -38.570764
2022-02-28 20:41:49.395208: Tratou 60760815: -3.8344742 - -38.564029
2022-02-28 20:41:55.608690: Tratou 60440125: -3.759635 - -38.5690628
2022-02-28 20:42:01.644915: Tratou 60813575: -3.7811416 - -38.4829603
2022-02-28 20:42:07.787316: Tratou 60125161: -3.7490242 - -38.5035487
2022-02-28 20:42:14.539598: Tratou 60430540: -3.7451526 - -38.5470536
2022-02-28 20:42:20.686450: Tratou 60130550: -3.7610456 - -38.5101854
2022-02-28 20:42:27.129724: Tratou 60125160: -3.732944 - -38.498373
2022-02-28 20:42:33.288062: Tratou 61618350: -3.6488441 - -38.69412
2022-02-28 20:42:39.432203: Tratou 60714070: -3.7830076 - -38.5571165
2022-02-28 20:42:46.780776:

 Salvando os dados em um arquivo.
 <p>É possível verificar que não foram retornadas coordenadas geográficas para uma pequena quantidade de CEPs. Estes CEPs serão descartados.

In [29]:
df_coords = pd.DataFrame({'cep':ceps,'lat':lats,'long':longs})
df_coords.dropna(inplace=True)
df_coords.head()

,cep,lat,long
0,60150060,-3.736327,-38.504833
1,60320105,-3.726792,-38.562826
2,60713480,-3.795657,-38.578317
3,60431075,-3.739664,-38.550117
4,60430660,-3.740088,-38.546496


In [31]:
df_coords.to_csv('arquivos/Coordenadas geográficas dos CEPs.csv',index=False)